In [4]:
from functions import *
from bokeh.embed import components

In [5]:
def getChart2(n2yokey=None, adhoc=None):
    userLat = 35 
    height = 500
    width = height*2
    issIndex = 6
    issDf = getISSList()
    northISSdf = projectDf(issDf, userLat, height)
    southISSdf = projectDf(issDf, -userLat,height)

    northDay = getDay(userLat,height)
    southDay = getDay(-userLat,height)
    northNight = getNight(userLat)
    southNight = getNight(-userLat)
    corners = getCorners()
    iss = getISS()
    hubble = getHubbleIcon()
    usa224 = getUSA224Icon()
    moonDf = getMoonList()
    northMoonDf = projectDf(moonDf, userLat, height).dropna().reset_index()
    southMoonDf = projectDf(moonDf, -userLat, height).dropna().reset_index()
    northMoon = getMoonLocation(userLat, height)
    southMoon = getMoonLocation(-userLat, height)
    moon = getMoon()
    footprint = int(2*math.sqrt(((abs(issDf.loc[issIndex]['footprint'])*width)/height)/math.pi))
    # footprint = int(2*math.sqrt((((issDf.loc[issIndex]['footprint']or 1)*width)/height)/math.pi))

    if not np.isnan(northISSdf.loc[issIndex]['x']):
        text = 'Lat:'+str(round(northISSdf.loc[issIndex]['lat'],2))\
                +' Lon:'+str(round(northISSdf.loc[issIndex]['lon'],2))\
                +'\r\nAltitude:'+str(round(northISSdf.loc[issIndex]['altitude'],2))+' '+str(northISSdf.loc[5]['units'])\
                +'\r\nSpeed:'+str(round(northISSdf.loc[issIndex]['velocity'],2))+' '+str(northISSdf.loc[5]['units']+' per hour')
    elif not np.isnan(southISSdf.loc[issIndex]['x']):
        text = 'Lat:'+str(round(southISSdf.loc[issIndex]['lat'],2))\
            +' Lon:'+str(round(southISSdf.loc[issIndex]['lon'],2))\
            +'\r\nAltitude:'+str(round(southISSdf.loc[issIndex]['altitude'],2))+' '+str(southISSdf.loc[5]['units'])\
            +'\r\nSpeed:'+str(round(southISSdf.loc[issIndex]['velocity'],2))+' '+str(southISSdf.loc[5]['units']+' per hour')
    else:
        text = ''

    c = figure(width = width, height = height, x_range =(-width, width), y_range=(-height,height))
    c.title.text = text
    c.title.align = "center"
    c.title.text_color = "white"
    c.title.text_font_size = "12px"
    c.title.background_fill_color = "black"

    ###NORTHERN HEMISPHERE
    c.image_rgba(image=[northNight], x =-width, y=-height, dh =width, dw=width)
    c.image_rgba(image=[northDay], x =-width, y=-height, dh =width, dw=width)

    ###SOUTHERN HEMISPHERE
    c.image_rgba(image=[southNight], x =0, y=-height, dh =width, dw=width)
    c.image_rgba(image=[southDay], x =0, y=-height, dh =width, dw=width)
    
    ###SATELLITES
    #ADHOC SAT NORTH AND SOUTH
    #ADHOC SATS i.e. NOAA19: 33591 ; GOES 15: 36411 ; LANDSAT 8: 39084; 
    adhocSat = adhoc
    if n2yokey and adhocSat:
        adHocNames =[]
        colorsAdhoc = viridis(len(adhocSat))
        if len(adhocSat)>0:
            for num, sat in enumerate(adhocSat, start = 0):
                colorIndex = num
                adf = getN2Y0sat(sat, n2yokey )
                adf = adf[adf.index%100 == 0].reset_index()
                northAdf = projectDf(adf, userLat, height)
                southAdf = projectDf(adf, -userLat, height)
                
                adHocNames.append(northAdf.loc[0]['satName'])
                plotSat(c, height, colorsAdhoc[colorIndex], northAdf, 0, 8, 25, hubble, 40)
                plotSat(c, -height, colorsAdhoc[colorIndex], southAdf, 0, 8, 25, hubble, 40)
                
    #ISS NORTH   
    plotSat(c, height, "purple", northISSdf, issIndex, 10, footprint, iss, 80, 0.5, 0.8)

    #MOON NORTH
    # c.line(northMoonDf.x-height, northMoonDf.y, color="white", line_dash=[5,15], line_width=1, line_alpha = 0.6)
    c.triangle(northMoonDf.x-height
        , northMoonDf.y
        , color="white"
        , size = 5
        , alpha = 0.2
        , angle = northMoonDf.bearingToNext - 1.5708 )
    if not np.isnan(northMoon.loc[0]['x']) and northMoon.loc[0]['x']<500:    
        c.image_rgba(image=[moon], x=northMoon.loc[0]['x']-height-75, y = northMoon.loc[0]['y']-75, dh =150, dw=150)

    c.image_rgba(image=[corners], x =-width, y=-height, dh =width, dw=width)
    
    #ISS SOUTH
    plotSat(c, -height, "purple", southISSdf, issIndex, 10, footprint, iss, 80, 0.5, 0.8)

    #MOON SOUTH
    # c.line(southMoonDf.x+height, southMoonDf.y, color="white", line_width=1, line_alpha = 0.6)#, line_dash=[5,15])    
    c.triangle(southMoonDf.x+height
        , southMoonDf.y
        , color="white"
        , size = 5
        , alpha = 0.2
        , angle = southMoonDf.bearingToNext - 1.5708 )
    if not np.isnan(southMoon.loc[0]['x']):
        c.image_rgba(image=[moon], x=southMoon.loc[0]['x']+height-75, y = southMoon.loc[0]['y']-75, dh =150, dw=150)

    c.image_rgba(image=[corners], x =0, y=-height, dh =width, dw=width)
    c.background_fill_color = "#000000"
    c.toolbar_location = None
    c.axis.visible = False
    c.border_fill_color = '#000000'
    c.outline_line_color = None

    #LEGEND
    c.circle(-960, 457, size=35, alpha = 0.5, color="purple")
    c.image_rgba(image=[iss], x=-997, y = 415, dh =80, dw=80)
    issLabel = Label(text = 'ISS', x = -910, y = 435, text_color = 'white', text_font_size = "10pt")
    c.add_layout(issLabel)

    if n2yokey and adhocSat:
        colorsAdhoc = viridis(len(adhocSat))
        if len(adhocSat)>0:
            for num, sat in enumerate(adhocSat, start = 0):
                colorIndex = num
                y = 455 #-num*70
                x = -830 + (num*170)
                circle = c.circle(x+20, y, color = colorsAdhoc[colorIndex], size=25, alpha = 0.9)
                image = c.image_rgba(image=[hubble], x= x, y = y-20, dh =40, dw=40)
                adHocLabel = Label(text = adHocNames[num], x = x+55, y = y-15, text_color = 'white', text_font_size = "8pt")
                c.add_layout(adHocLabel)


    # print (northMoon, southMoon)
    # output_notebook(hide_banner=True)
    # show(c)
    # end = datetime.now()-start
    # print(end, 'vs 2.78 seconds')
    return c

def getISSList2():
    now = datetime.timestamp(datetime.now())

    #start in the past 18 minutes
    stamp = int(now - (now%60)-1080)
    url = "https://api.wheretheiss.at/v1/satellites/25544/positions?timestamps="+ str(stamp)+","

    #append 35 more timestamps separated by 3 minutes each to the request URL
    for i in range(1,35):
        if i < 34:
            url = url + str(stamp+(180*i))+ ','
        else:
            url = url + str(stamp+(180*i))+ '&units=miles'
    response = requests.get(url)
    if response:
        # df = pd.read_json(response.content)
        df = pd.read_json(BytesIO(response.content))
        # df = pd.read_json(io.BytesIO(json.dumps(response.content, ensure_ascii=False).encode('utf8')), encoding='utf-8').to_csv(name, encoding="utf-8")
#     df = issDf[['latitude','longitude']]
    df.columns = ['name', 'id', 'lat', 'lon', 'altitude', 'velocity', 'visibility', 
                  'footprint', 'timestamp', 'daynum', 'solar_lat', 'solar_lon', 'units']
    return df

def getISSListDerp():
    now = datetime.timestamp(datetime.now())

    #start in the past 18 minutes
    stamp = int(now - (now%60)-1080)
    url = "https://api.wheretheiss.at/v1/satellites/25544/positions?timestamps="+ str(stamp)+","

    #append 35 more timestamps separated by 3 minutes each to the request URL
    for i in range(1,35):
        if i < 34:
            url = url + str(stamp+(180*i))+ ','
        else:
            url = url + str(stamp+(180*i))+ '&units=miles'
    response = requests.get(url)
    if response:
        # df = pd.read_json(response.content)
        df = pd.read_json(BytesIO(response.content))
        # df = pd.read_json(io.BytesIO(json.dumps(response.content, ensure_ascii=False).encode('utf8')), encoding='utf-8').to_csv(name, encoding="utf-8")
#     df = issDf[['latitude','longitude']]
    df.columns = ['name', 'id', 'lat', 'lon', 'altitude', 'velocity', 'visibility', 
                  'footprint', 'timestamp', 'daynum', 'solar_lat', 'solar_lon', 'units']
    return df

In [7]:
# components(getChart2(None,None))
issDf = getISSList()
# display(issDf)
northISSdf = projectDf(issDf, 35, 500)
# display(northISSdf)
print(str(round(northISSdf.loc[6]['altitude'],2)))

261.2
